# Libraries

In [1]:
import numpy as np
from scipy import stats
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

# Functions

In [2]:
#reads the XPS Data and gives meaninigful name to the columns. The colums must be 5
def ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns):
    XPS=pd.read_table(FolderTR+"/"+FileXPS, skiprows=lambda x: x%DataPerSpectrum<HeaderColumns, header=None)
    XPS.columns=["Kinetic energy", "Counts", "Initial Temperature", "Binding Energy", "CPS"]
    return XPS


#It computer the mean of a region of CPS (greater than a setpoint named lower energy or lower than a setpoint named upper energy) 
#and subtract it to to the data for each spectrum so that all the spectra cross the zero line.
def Normalize(XPS, NumberOfSpectra, NumberOfData):  
    
    x_len=XPS.shape[0]//NumberOfSpectra
    #reshape the data to compute the mean for each spectrum
    Means=np.mean(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[0:NumberOfData],axis=1)

    #subtratct the mean to each spectrum 
    for i in range (0, NumberOfSpectra):
        XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i]-= Means[i]
    
    return XPS

    
#For each spectrum, it computes the linear background and subtractit to the data
#To do so, it uses two doublets (x,y),
#they are the mean of the Point number of points at the right of the left edge and at the left of the rigth edge
def RemoveLinearBackgorund(XPS, Points, NumberOfSpectra):
    x_len=XPS.shape[0]//NumberOfSpectra

    for i in range (0, NumberOfSpectra):
        #for each spectrum compute the doublets (x,y) that will be used to calculate the slope m and intercept q
        y1=np.mean(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i][0:Points])
        y2=np.mean(XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i][x_len-Points:x_len])

        x1=np.mean(XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i][0:Points])
        x2=np.mean(XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i][x_len-Points:x_len])

        #compute slope and intercept
        m=(y2-y1)/(x2-x1)
        q=y2-m*x2

        #subtract to the CPS the as computed linear background
        XPS["CPS"].values.reshape(NumberOfSpectra,x_len)[i] -= m*XPS["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i] + q
        
    return XPS

# Main

In [58]:
FolderTR="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/0008_120221_AQ"
names=["5s Upstream", "5s Middle","300s Upstream", "300s Middle"]
colors=['#EF553B','#636EFA', '#AB63FA', '#00CC96']
symbol=["square","circle","cross","diamond"]

## HOPG

In [59]:
FileXPS="N1sHOPG" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=209 #number of rows of each spectrum including the headers
HeaderColumns=8 #number of columns comprising the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSHOPG=ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSHOPG.shape[0]//NumericDataPerSpectrum)

x_len=XPSHOPG.shape[0]//NumberOfSpectra


XPSHOPG=Normalize(XPSHOPG,NumberOfSpectra, NumberOfData=50)
XPSHOPG=RemoveLinearBackgorund(XPSHOPG, Points=50, NumberOfSpectra=NumberOfSpectra)


fig = go.Figure()


for i in range (0, NumberOfSpectra):
    fig.add_trace(go.Scatter(mode='markers', marker_symbol=symbol[i], marker=dict(color=colors[i]), x=XPSHOPG["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSHOPG["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=names[i]))
    
fig.add_vline(x=399, line_dash="dash", annotation_text="N-C")
fig.add_vline(x=400, line_dash="dash",annotation_text="N-O")
fig.add_vline(x=401.5, line_dash="dash", annotation_text="N+")

fig.update_layout(title="HOPG", xaxis_title='Binding Energy (eV)', yaxis_title='CPS',xaxis_range=(404,396))
    
fig.write_html(FolderTR+"/"+"HOPG"+".html")
fig.write_image(FolderTR+"/"+"HOPG"+".svg")

fig.show()

## Au

In [60]:
FolderTR="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/0003_020321_AQ"
FileXPS="N1sAu" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=209 #number of rows of each spectrum including the headers
HeaderColumns=8 #number of columns comprising the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSAu=ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSAu.shape[0]//NumericDataPerSpectrum)

x_len=XPSAu.shape[0]//NumberOfSpectra

#XPSAu=Normalize(XPSAu,NumberOfSpectra, NumberOfData=50)
XPSAu=RemoveLinearBackgorund(XPSAu, Points=50, NumberOfSpectra=NumberOfSpectra)



fig = go.Figure()



for i in range (0, NumberOfSpectra):
    fig.add_trace(go.Scatter(mode='markers', marker_symbol=symbol[i], x=XPSAu["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSAu["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=names[i]))
    
fig.add_vline(x=399, line_dash="dash", annotation_text="N-C")
fig.add_vline(x=400, line_dash="dash",annotation_text="N-O")
fig.add_vline(x=401.5, line_dash="dash", annotation_text="N+")

fig.update_layout(title="Au", xaxis_title='Binding Energy (eV)', yaxis_title='CPS',xaxis_range=(404,396))
    
fig.write_html(FolderTR+"/"+"Au"+".html")
fig.write_image(FolderTR+"/"+"Au"+".svg")

fig.show()

## Si

In [61]:
FolderTR="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/0003_020321_AQ"
FileXPS="N1sSi" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=209 #number of rows of each spectrum including the headers
HeaderColumns=8 #number of columns comprising the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSSi=ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSSi.shape[0]//NumericDataPerSpectrum)

x_len=XPSSi.shape[0]//NumberOfSpectra

XPSSi=RemoveLinearBackgorund(XPSSi, Points=50, NumberOfSpectra=NumberOfSpectra)



fig = go.Figure()



for i in range (0, NumberOfSpectra):
    fig.add_trace(go.Scatter(mode='markers', marker_symbol=symbol[i], x=XPSSi["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSSi["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=names[i]))
    
fig.add_vline(x=399, line_dash="dash", annotation_text="N-C")
fig.add_vline(x=400, line_dash="dash",annotation_text="N-O")
fig.add_vline(x=401.5, line_dash="dash", annotation_text="N+")

fig.update_layout(title="Si", xaxis_title='Binding Energy (eV)', yaxis_title='CPS',xaxis_range=(404,396))
    
fig.write_html(FolderTR+"/"+"Si"+".html")
fig.write_image(FolderTR+"/"+"Si"+".svg")

fig.show()

## PTFE

In [62]:
FolderTR="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/0003_020321_AQ"
FileXPS="N1sPTFE" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=209 #number of rows of each spectrum including the headers
HeaderColumns=8 #number of columns comprising the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSPTFE=ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSPTFE.shape[0]//NumericDataPerSpectrum)

x_len=XPSPTFE.shape[0]//NumberOfSpectra

XPSPTFE=RemoveLinearBackgorund(XPSPTFE, Points=50, NumberOfSpectra=NumberOfSpectra)


fig = go.Figure()


for i in range (0, NumberOfSpectra):
    fig.add_trace(go.Scatter(mode='markers', marker_symbol=symbol[i], x=XPSPTFE["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSPTFE["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=names[i]))
    
fig.add_vline(x=399, line_dash="dash", annotation_text="N-C")
fig.add_vline(x=400, line_dash="dash",annotation_text="N-O")
fig.add_vline(x=401.5, line_dash="dash", annotation_text="N+")

fig.update_layout(title="PTFE", xaxis_title='Binding Energy (eV)', yaxis_title='CPS',xaxis_range=(404,396))
    
fig.write_html(FolderTR+"/"+"PTFE"+".html")
fig.write_image(FolderTR+"/"+"PTFE"+".svg")

fig.show()

## Al

In [63]:
FolderTR="C:/Users/quadrell/OneDrive - Lancaster University/PhD Project/Data/XPS/0003_020321_AQ"
FileXPS="N1sAl" #Name of the file with XPS Data produced by CasaXPS Sav TAB Ascii
DataPerSpectrum=209 #number of rows of each spectrum including the headers
HeaderColumns=8 #number of columns comprising the headers
NumericDataPerSpectrum=DataPerSpectrum-HeaderColumns #number of rows with the numeric data

#Read the XPS data and assign them to a pandas dataframe
XPSAl=ReadXPSData(FolderTR, FileXPS, DataPerSpectrum, HeaderColumns)

#number of spectra composing the dataset
NumberOfSpectra=(XPSAl.shape[0]//NumericDataPerSpectrum)

x_len=XPSAl.shape[0]//NumberOfSpectra

XPSAl=RemoveLinearBackgorund(XPSAl, Points=50, NumberOfSpectra=NumberOfSpectra)


fig = go.Figure()



for i in range (0, NumberOfSpectra):
    fig.add_trace(go.Scatter(mode='markers', marker_symbol=symbol[i], x=XPSAl["Binding Energy"].values.reshape(NumberOfSpectra,x_len)[i], y=XPSAl["CPS"].values.reshape(NumberOfSpectra,x_len)[i], name=names[i]))
    
fig.add_vline(x=399, line_dash="dash", annotation_text="N-C")
fig.add_vline(x=400, line_dash="dash",annotation_text="N-O")
fig.add_vline(x=401.5, line_dash="dash", annotation_text="N+")

fig.update_layout(title="Al", xaxis_title='Binding Energy (eV)', yaxis_title='CPS',xaxis_range=(404,396))
    
fig.write_html(FolderTR+"/"+"Al"+".html")
fig.write_image(FolderTR+"/"+"Al"+".svg")

fig.show()